# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [19]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4179,) (1393,) (4179,) (1393,)


### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    unique_elements, counts = np.unique(spam_data['target'], return_counts=True)
    
    return counts[1]/(sum(counts))*100

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    longest = max(vect.get_feature_names(), key =len)
    return longest
    

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

def answer_three():
    #Vector
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    #Bayes
    MNBM = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predict_labels = MNBM.predict_proba(vect.transform(X_test))[:, 1]
    
    return roc_auc_score(y_test, predict_labels)
answer_three()

0.991545422134696

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    values = X_train_vectorized.max(0).toarray()[0]
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    return (pd.Series(values[sorted_tfidf_index[:20]],index=feature_names[sorted_tfidf_index[:20]]), 
        pd.Series(values[sorted_tfidf_index[:-21:-1]],index =feature_names[sorted_tfidf_index[:-21:-1]]))
answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
def answer_five():
    from sklearn.naive_bayes import MultinomialNB


    vectorizer = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_tfidf = vectorizer.transform(X_train)
    
    nb = MultinomialNB(alpha=0.1)
    
    nb.fit(X_train_tfidf, y_train)
    
    X_test_tfidf = vectorizer.transform(X_test)
    
    y_prob = nb.predict_proba(X_test_tfidf)[:, 1]
    auc_score = roc_auc_score(y_test, y_prob)
    return auc_score
answer_five()

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [11]:
def answer_six():
    spam_data1= spam_data
    spam_data1['length'] = spam_data1['text'].apply(len)
    avg_len_not_spam = spam_data1[spam_data1['target'] == 0]['length'].mean()
    avg_len_spam = spam_data1[spam_data1['target'] == 1]['length'].mean()

    return avg_len_not_spam , avg_len_spam
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [12]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [13]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import roc_auc_score
import numpy as np
from scipy.sparse import hstack
def answer_seven():
   
    vectorizer = TfidfVectorizer(min_df=5)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # Step 2: Calculate the length of each document (number of characters)
    X_train_len = np.array([len(doc) for doc in X_train]).reshape(-1, 1)
    
    # Step 3: Combine the TF-IDF features with the document length feature
    X_train_combined = hstack([X_train_tfidf, X_train_len])
    
    # Step 4: Fit the SVC model
    svc = SVC(C=10000, probability=False)
    svc.fit(X_train_combined, y_train)
    
    # Step 5: Transform the test data using the same vectorizer and document length calculation
    X_test_tfidf = vectorizer.transform(X_test)
    X_test_len = np.array([len(doc) for doc in X_test]).reshape(-1, 1)
    X_test_combined = hstack([X_test_tfidf, X_test_len])
    
    # Step 6: Compute the AUC score using the model's decision_function on the test data
    y_scores = svc.decision_function(X_test_combined)
    auc_score = roc_auc_score(y_test, y_scores)
    return auc_score
answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [14]:
def answer_eight():
    
    # Calculate number of digits per document
    spam_data1 = spam_data
    spam_data1['num_digits'] = spam_data1['text'].str.count(r'\d')
    
    # Calculate average number of digits for not spam (target == 0) and spam (target == 1)
    avg_digits_not_spam = spam_data1[spam_data1['target'] == 0]['num_digits'].mean()
    avg_digits_spam = spam_data1[spam_data1['target'] == 1]['num_digits'].mean()
    
    return (avg_digits_not_spam, avg_digits_spam)

answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [18]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range = (1,3))
    X_train_tfidf = vect.fit_transform(X_train)
    
    len1 = X_train.apply(len)
    len2 = X_test.apply(len)
    
    X_fin_train = add_feature(X_train_tfidf, [len1.values, X_train.str.count('\d')])
    X_fin_test = add_feature(vect.transform(X_test), [len2.values, X_test.str.count('\d')])
    clfrLR = LogisticRegression(C=100,max_iter=1000)
    
    clfrLR.fit(X_fin_train, y_train)
    
    predictions = clfrLR.predict_proba(X_fin_test)[:,1]
    return roc_auc_score(y_test,predictions)

answer_nine()

0.9973218681561211

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [16]:
def answer_ten():
    spam_data1 = spam_data
    spam_data1['num_charac'] = spam_data1['text'].str.count(r'\W')
    av_char_spam = spam_data1[spam_data1['target'] == 1]['num_charac'].mean()
    av_char_not = spam_data1[spam_data1['target'] == 0]['num_charac'].mean()
    return av_char_not, av_char_spam
    # YOUR CODE HERE
    raise NotImplementedError()
    return #Your answer here

answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

import re
from scipy.sparse import csr_matrix, hstack
def answer_eleven():
    # Use the first 2000 rows of training data
    X_train_subset = X_train[:2000]
    y_train_subset = y_train[:2000]
    
    # Initialize CountVectorizer with specified parameters
    vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 5), min_df=5)
    
    # Fit and transform X_train_subset
    X_train_counts = vectorizer.fit_transform(X_train_subset)
    
    # Transform X_test
    X_test_counts = vectorizer.transform(X_test)
    
    # Calculate additional features: length of document, number of digits, number of non-word characters
    X_train_lengths = pd.Series([len(text) for text in X_train_subset]).values.reshape(-1, 1)
    X_test_lengths = pd.Series([len(text) for text in X_test]).values.reshape(-1, 1)
    X_train_digits = pd.Series([sum(char.isdigit() for char in text) for text in X_train_subset]).values.reshape(-1, 1)
    X_test_digits = pd.Series([sum(char.isdigit() for char in text) for text in X_test]).values.reshape(-1, 1)
    X_train_non_word_chars = pd.Series([len(re.findall(r'\W', text)) for text in X_train_subset]).values.reshape(-1, 1)
    X_test_non_word_chars = pd.Series([len(re.findall(r'\W', text)) for text in X_test]).values.reshape(-1, 1)
    
    # Combine CountVectorizer matrix with additional features
    X_train_final = hstack([X_train_counts, X_train_lengths, X_train_digits, X_train_non_word_chars])
    X_test_final = hstack([X_test_counts, X_test_lengths, X_test_digits, X_test_non_word_chars])
    
    # Train Logistic Regression with specified parameters
    model = LogisticRegression(C=100, max_iter=1000)
    model.fit(X_train_final, y_train_subset)
    
    # Compute AUC score using decision_function on test data
    y_scores = model.decision_function(X_test_final)
    auc_score = roc_auc_score(y_test, y_scores)
    
    # Get feature names and include names for additional features
    feature_names = vectorizer.get_feature_names_out().tolist() + ['length_of_doc', 'digit_count', 'non_word_char_count']
    
    # Get model coefficients
    coefs = model.coef_.flatten()
    
    # Create a DataFrame to store feature names and their coefficients
    coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefs})
    
    # Find the 10 smallest and 10 largest coefficients
    smallest_coefs = coef_df.nsmallest(10, 'coefficient').sort_values(by='coefficient').reset_index(drop=True)
    largest_coefs = coef_df.nlargest(10, 'coefficient').sort_values(by='coefficient', ascending=False).reset_index(drop=True)
    
    # Convert to lists of tuples
    smallest_coefs_list = list(smallest_coefs['feature'])
    largest_coefs_list = list(largest_coefs['feature'])
    
    return (auc_score, smallest_coefs_list, largest_coefs_list)

# Call the function to get the result
answer_eleven()

(0.997568035583926,
 ['n ', ' i', 'at', 'he', ' m', '..', 'us', 'go', ' lo', ' bu'],
 ['digit_count', 'ne', ' st', 'co', 's ', 'xt', 'lt', 'xt ', ' ne', 'der'])